<a href="https://colab.research.google.com/github/RAGISHIVANAND/DEEP_LEARNING/blob/main/pytorch_Artificial_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Importing the necessary libraries***

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
#device configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper_parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate=0.001

In [5]:
#MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='inputdata',
                                           train=True,
                                           transform = transforms.ToTensor(),
                                           download=True)
test_dataset= torchvision.datasets.MNIST(root='inputdata',
                                           train=False,
                                           transform=transforms.ToTensor())
#Dataloader
train_loader=   torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 99480559.44it/s]


Extracting inputdata/MNIST/raw/train-images-idx3-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 42819262.57it/s]


Extracting inputdata/MNIST/raw/train-labels-idx1-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26523988.93it/s]


Extracting inputdata/MNIST/raw/t10k-images-idx3-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12467623.54it/s]


Extracting inputdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to inputdata/MNIST/raw



In [6]:
#FULLY CONNECTED NEURAL NETWORK WITH ONE HIDDEN LAYER
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2=nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out    

In [7]:
model = NeuralNet(input_size,hidden_size,num_classes).to(device)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
#Train the model
total_step = len(train_loader)
for epoch in range (num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        #Move tensors to the configured device
        images = images.reshape(-1,28*28).to(device)
        labels=labels.to(device)

        #Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1)% 100 == 0:
           print('Epoch[{}/{}],step[{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item())
                 .format(epoch+1,num_epochs,i+1,total_step,loss.item()))


Epoch[1/5],step[100/600],Loss:0.4721
Epoch[1/5],step[200/600],Loss:0.1471
Epoch[1/5],step[300/600],Loss:0.2417
Epoch[1/5],step[400/600],Loss:0.2828
Epoch[1/5],step[500/600],Loss:0.1591
Epoch[1/5],step[600/600],Loss:0.1848
Epoch[2/5],step[100/600],Loss:0.2583
Epoch[2/5],step[200/600],Loss:0.1116
Epoch[2/5],step[300/600],Loss:0.0782
Epoch[2/5],step[400/600],Loss:0.1067
Epoch[2/5],step[500/600],Loss:0.0437
Epoch[2/5],step[600/600],Loss:0.0585
Epoch[3/5],step[100/600],Loss:0.0412
Epoch[3/5],step[200/600],Loss:0.1692
Epoch[3/5],step[300/600],Loss:0.1125
Epoch[3/5],step[400/600],Loss:0.0350
Epoch[3/5],step[500/600],Loss:0.1071
Epoch[3/5],step[600/600],Loss:0.0558
Epoch[4/5],step[100/600],Loss:0.0869
Epoch[4/5],step[200/600],Loss:0.0510
Epoch[4/5],step[300/600],Loss:0.0707
Epoch[4/5],step[400/600],Loss:0.0458
Epoch[4/5],step[500/600],Loss:0.0123
Epoch[4/5],step[600/600],Loss:0.0322
Epoch[5/5],step[100/600],Loss:0.0268
Epoch[5/5],step[200/600],Loss:0.0597
Epoch[5/5],step[300/600],Loss:0.0174
E

In [10]:
#Test the model 
#in the test phase,we dont need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total +=labels.size(0)
        correct +=(predicted==labels).sum().item()
    print('Accuracy of the network on the 10000 test images:{}%'.format(100*correct/total))    

Accuracy of the network on the 10000 test images:97.91%
